In [1]:
import connectorx as cx
import pandas as pd
import numpy as np
import os
import duckdb
from datetime import datetime
import glob
import multiprocessing as mp
from functools import partial
import time

In [ ]:
def calculate_daily_factor(trading_date):
    order_pth = f"/data/HighFreqData/Order/l2order/{pd.to_datetime(trading_date).strftime('%Y%m%d')}.parquet"

    if not os.path.exists(order_pth):
        raise FileNotFoundError(f"订单数据文件不存在: {order_pth}")

    conn = duckdb.connect(database=':memory:')

    query_order = f"""
        WITH buy_orders_with_time AS(
            SELECT 
                security_code,
                order_side, 
                order_type,
                order_details,
                order_price,
                order_price_adj,
                order_volume,
                order_time,
                ('{trading_date}'::DATE + MAKE_TIME(
                    FLOOR(order_time / 10000000)::int,
                    (FLOOR(order_time / 100000) % 100)::int,
                    (FLOOR(order_time / 1000) % 100)::numeric + 
                    (order_time % 1000)::numeric / 1000)
                )::TIMESTAMP AS formatted_time    -- 拼接成TIME格式

            FROM '{order_pth}'
            WHERE order_side = 1
                AND order_type = 'A'
                AND order_details = 'L'
                AND order_price > 0
                AND order_volume > 0
                AND order_time >= 93000000
                AND order_time < 145700000
        )
        SELECT
            security_code,
            order_price,
            order_price_adj,
            order_volume,
            DATE_TRUNC('minute', formatted_time) AS order_minute
        FROM buy_orders_with_time
        """

    # 将数据加载到临时表中
    conn.execute(f"CREATE TEMPORARY TABLE minute_order AS {query_order}")

    APB_query = """
        WITH buy_orders_with_time AS(
            SELECT 
                security_code,
                order_minute,
                SUM(order_volume * order_price) AS minute_turnover_value,
                SUM(order_volume) AS minute_turnover_volume,
                SUM(order_volume * order_price)/SUM(order_volume) AS minute_vwap
            FROM minute_order
            GROUP BY security_code, order_minute
        )
        SELECT
            security_code,
            SUM(minute_turnover_value)/SUM(minute_turnover_volume) AS vwap,
            AVG(minute_vwap) AS twap
        FROM buy_orders_with_time
        GROUP BY security_code
    """

    conn.execute(f"CREATE TEMPORARY TABLE order_value AS {APB_query}")

    # 计算因子
    APB_Factors = conn.execute("""
    SELECT 
        security_code,
        CASE 
            WHEN twap > 0 AND vwap > 0 THEN LN(twap / vwap)
            ELSE NULL
        END AS new_APB
    FROM order_value
    """).fetchdf()

    return APB_Factors




In [4]:
def derive_daily_factor (start_date, end_date, factor_name):
    
    #交易日列表
    trading_dates = pd.date_range(start=start_date, end=end_date, freq='B') 
    output_dir = f"./factors/{factor_name}"
    os.makedirs(output_dir, exist_ok=True)
    # 记录处理结果
    processed_dates = []
    skipped_dates = []
    
    for trading_date in trading_dates:
        date_str = trading_date.strftime('%Y%m%d')
        file_path = f"{output_dir}/{date_str}.parquet"
        
        # 如果文件已存在，跳过处理
        if os.path.exists(file_path):
            print(f"文件已存在，跳过: {file_path}")
            processed_dates.append(date_str)
            continue
        
        # 检查原始数据文件是否存在
        order_pth = f"/data/HighFreqData/Order/l2order/{date_str}.parquet"

        if not os.path.exists(order_pth):
            print(f"跳过日期 {date_str}: 订单数据文件不存在")
            skipped_dates.append(date_str)
            continue
        
        try:
            # 计算因子
            daily_factor_df = calculate_daily_factor(trading_date)
            
            # 添加日期列
            daily_factor_df['date'] = date_str
            
            # 保存结果
            daily_factor_df.to_parquet(file_path, index=False)
            print(f"已生成因子文件: {file_path}")
            processed_dates.append(date_str)
            
        except Exception as e:
            print(f"处理日期 {date_str} 时出错: {str(e)}")
            skipped_dates.append(date_str)
    
    # 打印处理结果摘要
    print(f"\n处理完成:")
    print(f"成功处理 {len(processed_dates)} 个日期")
    print(f"跳过 {len(skipped_dates)} 个日期")
    
    return output_dir

In [5]:
def calculate_rolling_20d_avg(factor_names, factor_dir, start_date=None, end_date=None, min_window=5):
    
    if not isinstance(factor_names, list):
        factor_names = [factor_names]
    
    parquet_pattern = os.path.join(factor_dir, "*.parquet")
    parquet_files = glob.glob(parquet_pattern)
    print(f"在目录 {factor_dir} 中找到 {len(parquet_files)} 个Parquet文件")
    

    conn = duckdb.connect(database=':memory:')
    all_factor_data = conn.execute(f"""
        SELECT * FROM read_parquet('{parquet_pattern}')
    """).fetchdf()
    conn.close()
    

    missing_factors = [f for f in factor_names if f not in all_factor_data.columns]
    if missing_factors:
        raise ValueError(f"在Parquet文件中未找到以下因子列: {', '.join(missing_factors)}")
    
    all_factor_data['date'] = pd.to_datetime(all_factor_data['date'])

    all_factor_data = all_factor_data.sort_values(['date', 'security_code'])
    
    # 筛选日期范围
    if start_date:
        start_date = pd.to_datetime(start_date)
        all_factor_data = all_factor_data[all_factor_data['date'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date)
        all_factor_data = all_factor_data[all_factor_data['date'] <= end_date]

    # 获取所有个股代码
    all_securities = all_factor_data['security_code'].unique()

    result_df = all_factor_data[['date', 'security_code']].copy()

    # 为每个因子计算滚动平均
    for factor_name in factor_names:
        print(f"\n处理因子: {factor_name}")
        
        #转置成宽表
        pivot_data = all_factor_data.pivot(index='date', columns='security_code', values=factor_name)

        # 对宽表直接应用rolling
        rolling_avg = pivot_data.rolling(window=20, min_periods=min_window).mean()

        # 将结果转换回长格式
        factor_df = rolling_avg.stack().reset_index()
        factor_df.columns = ['date', 'security_code', factor_name]
        # factor_df[factor_name] = - factor_df[factor_name]

        # 计算每日因子覆盖率
        # 计算每个日期非NaN的因子值数量
        non_nan_counts = factor_df.dropna(subset=[factor_name]).groupby('date').size()

        # 计算覆盖率
        coverage = non_nan_counts / len(all_securities)
        print(f"\n{factor_name}因子覆盖率统计: 平均={coverage.mean():.2f}, 最小={coverage.min():.2f}")

        result_df = pd.merge(
            result_df, 
            factor_df,
            on=['date', 'security_code'],
            how='left'
        )
    
    return result_df

In [6]:
def process_single_date(trading_date, factor_name):
    
    """处理单个交易日的函数"""
   
    date_str = trading_date.strftime('%Y%m%d')
    output_dir = f"./factors/{factor_name}"
    file_path = f"{output_dir}/{date_str}.parquet"
    
    # 如果文件已存在，跳过处理
    if os.path.exists(file_path):
        print(f"文件已存在，跳过: {file_path}")
        return date_str, True, "已存在"
    
    # 检查原始数据文件是否存在
    data_file = f"/data/HighFreqData/Order/l2order/{date_str}.parquet"
    if not os.path.exists(data_file):
        print(f"跳过日期 {date_str}: 原始数据文件不存在")
        return date_str, False, "数据文件不存在"
    
    try:
        daily_factor_df = calculate_daily_factor(trading_date)
        
        daily_factor_df['date'] = date_str

        daily_factor_df.to_parquet(file_path, index=False)
        print(f"已生成因子文件: {file_path}")
        return date_str, True, "成功"
        
    except Exception as e:
        error_msg = str(e)
        print(f"处理日期 {date_str} 时出错: {error_msg}")
        return date_str, False, error_msg


def derive_daily_factor(start_date, end_date, factor_name, num_processes=30):
    start_time = time.time()
    
    trading_dates = pd.date_range(start=start_date, end=end_date, freq='B')
    
    # 创建输出目录
    output_dir = f"./factors/{factor_name}"
    os.makedirs(output_dir, exist_ok=True)
    
    # 创建进程池
    pool = mp.Pool(processes=num_processes)
    
    # 创建带有固定参数的处理函数
    process_date_with_args = partial(process_single_date, factor_name=factor_name)
    
    # 提交所有任务到进程池并获取结果
    print(f"开始使用 {num_processes} 个进程并行处理 {len(trading_dates)} 个交易日...")
    results = pool.map(process_date_with_args, trading_dates)
    
    # 关闭进程池
    pool.close()
    pool.join()
    
    # 分析处理结果
    processed_dates = [date_str for date_str, success, _ in results if success]
    skipped_dates = [date_str for date_str, success, _ in results if not success]
    
    # 打印处理结果摘要
    print(f"\n处理完成:")
    print(f"成功处理 {len(processed_dates)} 个日期")
    print(f"跳过 {len(skipped_dates)} 个日期")

    return output_dir


In [10]:
def main():

    start_date = '2021-05-18'
    end_date = '2024-12-31'

    factor_name = "raw_order_APB"
    
    factor_dir = derive_daily_factor (start_date, end_date, factor_name)
    print(f"所有因子文件已生成在目录: {factor_dir}")

    factor_name = ['new_APB']

    result_df = calculate_rolling_20d_avg(factor_name, factor_dir, start_date, end_date, min_window=5)
    
    # 保存结果为CSV
    output_path = "/data/home/lexuanchen/Factors/Order/Signal/Improved_APB"
    os.makedirs(output_path, exist_ok=True)

    result_df.to_csv(f'{output_path}/raw_order_APB.csv')

    print(f"共计 {len(result_df)} 条记录")

    print("\n数据预览:")
    print(result_df.head())

# 执行主函数
if __name__ == "__main__":
    main()


文件已存在，跳过: ./factors/raw_order_APB/20210518.parquet文件已存在，跳过: ./factors/raw_order_APB/20210528.parquet文件已存在，跳过: ./factors/raw_order_APB/20210609.parquet文件已存在，跳过: ./factors/raw_order_APB/20210713.parquet文件已存在，跳过: ./factors/raw_order_APB/20210621.parquet文件已存在，跳过: ./factors/raw_order_APB/20210701.parquet
文件已存在，跳过: ./factors/raw_order_APB/20210723.parquet文件已存在，跳过: ./factors/raw_order_APB/20210804.parquet文件已存在，跳过: ./factors/raw_order_APB/20210826.parquet文件已存在，跳过: ./factors/raw_order_APB/20210816.parquet

文件已存在，跳过: ./factors/raw_order_APB/20210929.parquet文件已存在，跳过: ./factors/raw_order_APB/20210907.parquet文件已存在，跳过: ./factors/raw_order_APB/20211011.parquet文件已存在，跳过: ./factors/raw_order_APB/20211102.parquet
文件已存在，跳过: ./factors/raw_order_APB/20211112.parquet
文件已存在，跳过: ./factors/raw_order_APB/20210519.parquet
文件已存在，跳过: ./factors/raw_order_APB/20211124.parquet文件已存在，跳过: ./factors/raw_order_APB/20211216.parquet文件已存在，跳过: ./factors/raw_order_APB/20220107.parquet
跳过日期 20211228: 原始数据文件不存在文件已存在，跳过: ./factors

In [12]:
output_path = "/data/home/lexuanchen/Factors/Order/Signal/Improved_APB"
df = pd.read_csv(f'{output_path}/raw_order_APB.csv')
df.rename(columns={'new_APB': 'raw_order_APB'}, inplace=True)

df.to_csv(f'{output_path}/raw_order_APB.csv')


stats = df['raw_order_APB'].describe()
print(stats)

count    4.127222e+06
mean     1.691010e-04
std      1.368705e-02
min     -1.974252e-02
25%     -9.191939e-04
50%     -2.354157e-04
75%      3.001033e-04
max      2.185038e+00
Name: raw_order_APB, dtype: float64
